<a href="https://colab.research.google.com/github/shashithenuwara/FDM-MINI_project/blob/main/Support_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
df_before = pd.read_csv("Sleep_Data_Sampled.csv")

In [3]:
df = df_before.drop("Person ID",axis = 1)

In [4]:
df.head(20)

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Fatigue Score,Sleep Disorder
0,Male,35,Doctor,6.65,7,50,7,Under Weight,120/80,71,7100,7.52,Healthy
1,Male,42,Teacher,6.90,8,52,4,Normal,135/90,66,7000,7.54,Healthy
2,Male,34,Software Engineer,6.95,7,66,6,Overweight,126/83,74,6100,9.50,Healthy
3,Male,32,Doctor,6.90,6,52,7,Normal,120/80,71,6500,7.54,Healthy
4,Male,37,Lawyer,6.85,7,60,6,Normal,125/80,71,6500,8.76,Healthy
5,Male,33,Doctor,6.90,7,50,6,Under Weight,120/80,69,7500,7.25,Healthy
6,Male,44,Engineer,7.85,8,45,4,Normal,125/80,65,5000,5.73,Healthy
7,Male,40,Engineer,8.05,8,52,4,Normal,120/80,68,6500,6.46,Healthy
8,Male,36,Lawyer,6.85,7,60,6,Normal,130/85,71,6500,8.76,Healthy
9,Male,40,Engineer,7.15,7,58,6,Under Weight,130/85,70,6500,8.11,Healthy


**1.Data Cleaning**

**Handle Missing Values**(Using Global Constant)

In [5]:
# Replace missing values in numeric columns with 0
df.fillna({col: 0 for col in df.select_dtypes(include='number').columns}, inplace=True)

# Verify if missing values are handled
df.isnull().sum()


,0
Gender,0
Age,0
Occupation,0
Sleep Duration,0
Quality of Sleep,0
Physical Activity Level,0
Stress Level,0
BMI Category,0
Blood Pressure,0
Heart Rate,0


**Handle Noisy Data (Replace minus valuse in Fatigue Score coloumn with mean value)**

In [6]:
# Calculate the mean of valid (non-negative) Fatigue Score values
mean_fatigue_score = df[df['Fatigue Score'] >= 0]['Fatigue Score'].mean()

# Replace negative values in Fatigue Score with the mean value
df['Fatigue Score'] = df['Fatigue Score'].apply(lambda x: mean_fatigue_score if x < 0 else x)

# Verify changes
df['Fatigue Score'].describe()


,Fatigue Score
count,15000.000000
mean,8.004071
std,3.150786
min,0.000000
25%,6.820000
50%,7.730000
75%,9.930000
max,15.250000


**Handle Non-Numerical Valus**

In [7]:
def convert_blood_pressure(bp):
    try:

        systolic, diastolic = map(int, bp.split('/'))
        return (systolic + diastolic) / 2  # Example: return the average
    except:
        return None

In [8]:
df['Blood Pressure'] = df['Blood Pressure'].apply(convert_blood_pressure)

In [9]:
df.head(55)

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Fatigue Score,Sleep Disorder
0,Male,35,Doctor,6.65,7,50,7,Under Weight,100.0,71,7100,7.520000,Healthy
1,Male,42,Teacher,6.90,8,52,4,Normal,112.5,66,7000,7.540000,Healthy
2,Male,34,Software Engineer,6.95,7,66,6,Overweight,104.5,74,6100,9.500000,Healthy
3,Male,32,Doctor,6.90,6,52,7,Normal,100.0,71,6500,7.540000,Healthy
4,Male,37,Lawyer,6.85,7,60,6,Normal,102.5,71,6500,8.760000,Healthy
5,Male,33,Doctor,6.90,7,50,6,Under Weight,100.0,69,7500,7.250000,Healthy
6,Male,44,Engineer,7.85,8,45,4,Normal,102.5,65,5000,5.730000,Healthy
7,Male,40,Engineer,8.05,8,52,4,Normal,100.0,68,6500,6.460000,Healthy
8,Male,36,Lawyer,6.85,7,60,6,Normal,107.5,71,6500,8.760000,Healthy
9,Male,40,Engineer,7.15,7,58,6,Under Weight,107.5,70,6500,8.110000,Healthy


**2.Data Transformation**

In [10]:
# Selecting the numeric columns for normalization
numeric_columns = ['Sleep Duration', 'Quality of Sleep', 'Physical Activity Level',
                   'Stress Level', 'Heart Rate', 'Daily Steps', 'Fatigue Score']

**Get Numeric Values to 0-10 Scale**

In [11]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 10))



In [12]:
# Apply the scaler to the numeric columns

df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

df[numeric_columns] = df[numeric_columns].round(2)


In [13]:
#View the Normalized Data
df.head(55)

,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Fatigue Score,Sleep Disorder
0,Male,35,Doctor,3.15,6.0,3.33,8.0,Under Weight,100.0,2.86,5.86,4.93,Healthy
1,Male,42,Teacher,4.07,8.0,3.67,2.0,Normal,112.5,0.48,5.71,4.94,Healthy
2,Male,34,Software Engineer,4.26,6.0,6.00,6.0,Overweight,104.5,4.29,4.43,6.23,Healthy
3,Male,32,Doctor,4.07,4.0,3.67,8.0,Normal,100.0,2.86,5.00,4.94,Healthy
4,Male,37,Lawyer,3.89,6.0,5.00,6.0,Normal,102.5,2.86,5.00,5.74,Healthy
5,Male,33,Doctor,4.07,6.0,3.33,6.0,Under Weight,100.0,1.90,6.43,4.75,Healthy
6,Male,44,Engineer,7.59,8.0,2.50,2.0,Normal,102.5,0.00,2.86,3.76,Healthy
7,Male,40,Engineer,8.33,8.0,3.67,2.0,Normal,100.0,1.43,5.00,4.24,Healthy
8,Male,36,Lawyer,3.89,6.0,5.00,6.0,Normal,107.5,2.86,5.00,5.74,Healthy
9,Male,40,Engineer,5.00,6.0,4.67,6.0,Under Weight,107.5,2.38,5.00,5.32,Healthy


**3.Data Mining**

**Encode Categorical Variables using One-Hot Encoding**

In [14]:
from sklearn.preprocessing import LabelEncoder

# Encode categorical columns
label_encoder = LabelEncoder()

df['Encoded_Column_Gender'] = label_encoder.fit_transform(df['Gender'])
df['Encoded_Column_Occupation'] = label_encoder.fit_transform(df['Occupation'])
df['Encoded_Column_BMI Category'] = label_encoder.fit_transform(df['BMI Category'])
#df['Encoded_Column_Sleep Disorder'] = label_encoder.fit_transform(df['Sleep Disorder'])


In [15]:
df = pd.get_dummies(df, columns=['Gender'],)
df = pd.get_dummies(df, columns=['Occupation'])
df = pd.get_dummies(df, columns=['BMI Category'])
#df = pd.get_dummies(df, columns=['Sleep Disorder'])

In [16]:
df.head(5)

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Blood Pressure,Heart Rate,Daily Steps,Fatigue Score,Sleep Disorder,...,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal,BMI Category_Obese,BMI Category_Overweight,BMI Category_Under Weight
0,35,3.15,6.0,3.33,8.0,100.0,2.86,5.86,4.93,Healthy,...,False,False,False,False,False,False,False,False,False,True
1,42,4.07,8.0,3.67,2.0,112.5,0.48,5.71,4.94,Healthy,...,False,False,False,False,False,True,True,False,False,False
2,34,4.26,6.0,6.00,6.0,104.5,4.29,4.43,6.23,Healthy,...,False,False,False,False,True,False,False,False,True,False
3,32,4.07,4.0,3.67,8.0,100.0,2.86,5.00,4.94,Healthy,...,False,False,False,False,False,False,True,False,False,False
4,37,3.89,6.0,5.00,6.0,102.5,2.86,5.00,5.74,Healthy,...,False,False,False,False,False,False,True,False,False,False


**Split dataset into Features(X) and Target(y)**

In [17]:
# Define the feature matrix (X) and the target vector (y)
X = df.drop('Sleep Disorder', axis=1)
y = df['Sleep Disorder']


**Train-Test Split**

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Support Vector Machine Classifier**

In [19]:
from sklearn.svm import SVC

# Initialize the Decision Tree model
svm = SVC(random_state=42)

# Train the model
svm.fit(X_train, y_train)




SVC(random_state=42)

In [20]:
from sklearn.metrics import accuracy_score, f1_score

# Predict on the test set
y_pred = svm.predict(X_test)

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')

Accuracy: 0.8943333333333333
F1 Score: 0.8939625838736031


In [21]:
from sklearn.metrics import confusion_matrix

# Compute confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[973  21  32]
 [ 83 854  60]
 [ 56  65 856]]


In [22]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

     Healthy       0.88      0.95      0.91      1026
    Insomnia       0.91      0.86      0.88       997
 Sleep Apnea       0.90      0.88      0.89       977

    accuracy                           0.89      3000
   macro avg       0.90      0.89      0.89      3000
weighted avg       0.90      0.89      0.89      3000

